In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import forestplot as fp
import statsmodels.api as sm
import statsmodels.stats.power as smp
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan

import import_ipynb
from apriori_power_analysis import calculate_target_cohens_f

In [ ]:
def linear_regression_assumption_checks(df, independent_var_col, dependent_var_col, robust_std_errors=False):
    """
    Perform assumption checks for a multiple linear regression model.
    
    Parameters:
    dataset (pd.DataFrame): The input dataset containing the dependent and independent variables.
    
    Returns:
    None
    """
    # --- 1. Prepare the Data ---
    print("\n--- Preparing Data for Linear Regression ---")

    y = df[dependent_var_col]
    X = df[independent_var_col]
    X = sm.add_constant(X) # Add a constant (intercept) to the model


    # --- 2. Fit the Multiple Linear Regression Model ---
    if robust_std_errors:
        print("\nFitting model with robust standard errors (HC3)...")
        model = sm.OLS(y, X).fit(cov_type='HC3')
    else:
        print("\nFitting model with standard OLS...")
    model = sm.OLS(y, X).fit()

    print(model.summary())

    # Extract fitted values and residuals
    fitted_vals = model.fittedvalues
    residuals = model.resid

    # --- 3. Check for Linearity & 4. Check for Homoscedasticity ---
    # These can be checked together using the Residuals vs. Fitted plot.

    print("\n--- 1. Checking Linearity and 2. Homoscedasticity ---")
    plt.figure(figsize=(8, 5))
    sns.residplot(x=fitted_vals, y=residuals, lowess=True,
                line_kws={'color': 'red', 'lw': 2},
                scatter_kws={'alpha': 0.5})
    plt.title('Residuals vs. Fitted Values')
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.grid(True)
    plt.show()

    print("Interpretation of Residuals vs. Fitted Plot:")
    print("1. Linearity: The red line should be close to horizontal at 0. A curve indicates non-linearity.")
    print("2. Homoscedasticity: The points should be randomly scattered in a constant-width band. A funnel shape indicates heteroscedasticity.")


    # Check Linearity with Partial Regression Plots
    print("\n--- ASSUMPTION 1: Linearity with Partial Regression Plots ---")
    fig = plt.figure(figsize=(12, 8))
    sm.graphics.plot_partregress_grid(model, fig=fig)
    fig.suptitle("Partial Regression Plots", fontsize=16, y=1.02)
    plt.show()

    print("Interpretation: Each plot shows the relationship between y and one predictor,")
    print("holding other predictors constant. Look for a linear trend in each subplot.")


    # Homoscedasticity Test
    print("\n--- ASSUMPTION 2: Homoscedasticity Test ---")
    bp_stat2, bp_p2, _, _ = het_breuschpagan(model.resid, X)
    HOMOSCEDASTICITY_GOOD = "✅" if bp_p2 > 0.05 else "❌"
    print(f"{HOMOSCEDASTICITY_GOOD} Breusch-Pagan Test: statistic={bp_stat2:.4f}, p-value={bp_p2:.4f}")
    print("Interpretation of Breusch-Pagan Test:")
    print(" - p > 0.05 suggests homoscedasticity (constant variance)")

    # ---  Check for Independence of Residuals ---
    print("\n--- ASSUMPTION 3: Checking Independence of Residuals (Autocorrelation) ---")
    WATSON_GOOD = "✅" if 1.5 < durbin_watson(model.resid) < 2.5 else "❌"
    durbin_watson_stat = sm.stats.durbin_watson(model.resid)
    print(f"{WATSON_GOOD} Durbin-Watson statistic: {durbin_watson_stat:.2f}")
    print("Interpretation: The statistic ranges from 0 to 4.")
    print("- A value around 2.0 suggests no autocorrelation.")
    print("- Values < 1.5 suggest positive autocorrelation.")
    print("- Values > 2.5 suggest negative autocorrelation.")

    # --- Check for Normality of Residuals ---
    print("\n--- ASSUMPTION 4: Checking Normality of Residuals ---")
    shapiro_stat2, shapiro_p2 = stats.shapiro(model.resid)
    SHAPIRO_GOOD = "✅" if shapiro_p2 > 0.05 else "❌"
    print(f"{SHAPIRO_GOOD} Shapiro-Wilk Test: statistic={shapiro_stat2:.4f}, p-value={shapiro_p2:.4f}")
    print("\nInterpretation of Shapiro-Wilk Test:")
    print(" - p > 0.05 suggests residuals are normally distributed")

    # plot these side by side
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))

    # Histogram
    sns.histplot(residuals, kde=True, ax=axs[0])
    axs[0].set_title('Histogram of Residuals')
    axs[0].set_xlabel('Residual')
    axs[0].set_ylabel('Frequency')
    axs[0].grid(True)

    # Q-Q Plot
    sm.qqplot(residuals, line='45', fit=True, ax=axs[1])
    axs[1].set_title('Q-Q Plot of Residuals')
    plt.show()

    print("Interpretation:")
    print("1. Histogram: Should look like a bell curve.")
    print("2. Q-Q Plot: Points should fall along the 45-degree line.")


    # --- Check for Multicollinearity ---
    print("\n--- ASSUMPTION 5: Checking for Multicollinearity ---")

    # Create a DataFrame for VIF
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # We ignore the VIF for the constant, as it's not a predictor.
    for row, i in vif_data[1:].iterrows():
        VIF_GOOD = "✅" if i['VIF'] < 5 else "❌"
        print(f"{VIF_GOOD} VIF for {i['feature']}: {i['VIF']:.4f}")
    print("\nInterpretation of VIF:")
    print("- VIF = 1: No correlation")
    print("- 1 < VIF < 5: Moderate correlation, generally acceptable.")
    print("- VIF > 5 or 10: High correlation, indicating a multicollinearity problem.")

In [ ]:
def build_labelmaps():
    # Let's build nice Label Maps
    labelmaps = {}
    labelmaps['control_treatment_asnumber'] = "1= Treatment Group"
    labelmaps['first_prog_course_asnumber'] = "1= First Programming Course"
    labelmaps['gender_asnumber'] = "1= Male"
    labelmaps['year_in_school_simplified_asnumber'] = "1= Not First Year"
    labelmaps['learning_session_count'] = "Learning Session Count"
    labelmaps['task_completion_session_count'] = "Task Completion Session Count"
    labelmaps['total_interaction_count'] = "Total Interactions"
    labelmaps['total_session_count'] = "Total Sessions"
    return labelmaps

def forest_plot_regression(data, labelmaps: dict = {}):
    """
    Create a forest plot from a statsmodels regression result object.

    Parameters:
    data (statsmodels.regression.linear_model.RegressionResultsWrapper): The regression result object.

    Returns:
    None: Displays the forest plot.
    """
    # Ensure the input is a statsmodels regression result object
    if not isinstance(data, sm.regression.linear_model.RegressionResultsWrapper):
        raise ValueError("Input must be a statsmodels regression result object.")


    # Create a dataframe with independent variable statistics from the regression model
    independent_vars_stats = pd.DataFrame({
        'Variable': [i for i in data.params.index[1:]],
        'Coefficient': data.params.values[1:],
        'Std_Error': data.bse.values[1:],
        't_statistic': data.tvalues.values[1:],
        'p_value': data.pvalues.values[1:],
        'CI_Lower': [data.conf_int().loc[i, 0] for i in data.params.index[1:]],
        'CI_Upper': [data.conf_int().loc[i, 1] for i in data.params.index[1:]],
        'N': [int(data.nobs) for _ in data.params.index[1:]],
        'Label': [labelmaps.get(i, i) if labelmaps else i for i in data.params.index[1:]],
        'Group': [ 'AI Interaction' if l in ["learning_session_count", "task_completion_session_count"] else 'Other' for l in data.params.index[1:]],  # No grouping in this case
    })

    return fp.forestplot(independent_vars_stats,  # the dataframe with results data
                estimate="Coefficient",  # col containing estimated effect size 
                ll="CI_Lower", hl="CI_Upper",  # columns containing conf. int. lower and higher limits
                varlabel="Label",  # column containing variable label
                groupvar="Group", # column containing group label (if any)
                ylabel=f"Confidence intervals for dependent variable {data.model.endog_names}",  # y-label title
                xlabel="Coefficient",  # x-label title
                pval="p_value",  # column containing p-values
                annote=["N",'t_statistic', "est_ci"],  # additional columns to display in the plot
                annoteheaders=["N", "T-Statistic", "Est. (95% Conf. Int.)"], 
                color_alt_rows=len(data.params.index[1:]) > 2,  # Gray alternate rows,
                table=True,
                )
    

def linear_regression_analysis(df, independent_var_col, dependent_var_col, do_print=True, do_plot=True, robust_std_errors=False):
    # 1. threshhold, based on apriori power analysis N=87
    if isinstance(independent_var_col, str):
        k_regressors = 1
    else:
        k_regressors = len(independent_var_col)
    N = df.shape[0]
    result = calculate_target_cohens_f(N, k_regressors)
    TARGET_F = result['cohens_f']
    TARGET_F2 = TARGET_F**2
    TARGET_R2 = TARGET_F2 / (1 + TARGET_F2) 

    # 2. Define the independent (X) and dependent (y) variables
    # We add a constant to the independent variable to fit the intercept (beta_0)
    # of the regression model.
    X = sm.add_constant(df[independent_var_col])  # Independent variable
    y = df[dependent_var_col]

    # 3. Fit the regression model, calculate the other vars 
    if robust_std_errors:
        print("\nFitting model with robust standard errors (HC3)...")
        model = sm.OLS(y, X).fit(cov_type='HC3')
    else:
        print("\nFitting model with standard OLS...")
        model = sm.OLS(y, X).fit()
        
    PGOOD = "✅" if model.f_pvalue < 0.05 else "❌"
    RGOOD = "✅" if model.rsquared >= TARGET_R2 else "❌"
    cohens_f2 = model.rsquared / (1 - model.rsquared)

    # 4. Print the regression summary
    if do_print:
        print("*" * 80)
        print("INDEPENDENT VAR:", independent_var_col) 
        print("DEPENDENT VAR  :", dependent_var_col)
        print("REGRESSORS     :", k_regressors)
        print(model.summary())
        print("Statistical Significance and Effect Size Checks:")
        print(f"{PGOOD} P-value : {model.f_pvalue:.4f} (Threshold: <= 0.05)")
        print(f"{RGOOD} R-squared: {model.rsquared:.4f} (Target R-Squared: >= {TARGET_R2:.4f})")
        print(f"   Cohen's f-squared: {cohens_f2:.4f} (Cohen's target f-squared: >= {TARGET_F2:.4f})")
        print(f"   Cohen's f: {np.sqrt(cohens_f2):.4f} (Cohen's target f: >= {TARGET_F:.4f})")

    if do_plot:
        display(forest_plot_regression(model, labelmaps=build_labelmaps()))

        
    return model



In [2]:

def convert_d_to_f(cohen_d: float) -> float:
    """
    Converts Cohen's d to Cohen's f for a two-group comparison.

    Cohen's d is an effect size used to indicate the standardized
    difference between two means. Cohen's f is an effect size used in the
    context of ANOVA. For the specific case of two groups (which is the
    context for Cohen's d), there is a direct conversion formula.

    The formula is: f = d / 2.

    This function takes the absolute value of d, as effect sizes are
    conventionally reported as non-negative values representing the
    magnitude of an effect.

    Args:
        cohen_d (float): The value of Cohen's d.

    Returns:
        float: The corresponding value of Cohen's f.

    Raises:
        TypeError: If the input cohen_d is not a numeric value.
    """
    if not isinstance(cohen_d, numbers.Number):
        raise TypeError("Input 'cohen_d' must be a numeric value (int or float).")

    # The conversion formula for two groups is f = d / 2.
    # Effect sizes are conventionally positive, so we take the absolute value.
    cohen_f = abs(cohen_d) / 2

    return cohen_f

def interpret_effect_size_cohens( effect_size: float) -> str:
    """
    Interpret the effect size based on power analysis thresholds.

    Args:
        effect_size (float): The calculated eta-squared value.
        
    Returns:
        str: Interpretation of the effect size.
    """
    if np.isnan(effect_size):
        return "Effect size is not calculable."
    
    effect_size = abs(effect_size)
    if effect_size < .50:
        interpretation  = "insignificant 👎"
    elif effect_size < 0.65:
        interpretation  = "in target range 🎯"
    else:
        interpretation  = "satisfactory 🚀"

    return interpretation 

def interpret_cohens_d(d):
    """Interprets the magnitude of Cohen's d."""
    d = abs(d)
    if d < 0.2:
        return "Very Small"
    elif d < 0.5:
        return "Small"
    elif d < 0.8:
        return "Medium"
    else:
        return "Large"

def interpret_rank_biserial(r):
    """Interprets the magnitude of Rank-Biserial Correlation."""
    r = abs(r)
    if r < 0.1:
        return "Very Small"
    elif r < 0.3:
        return "Small"
    elif r < 0.5:
        return "Medium"
    else:
        return "Large"

def independent_ttest(df: pd.DataFrame, independent_var_df_col: str, dependent_var_df_col: str):
    '''
    Perform an independent samples t-test (A/B test) on the specified dependent variable 
    grouped by the independent variable. The independent variable must have exactly two categories.

    Independent variable is the cause, should be categorical (e.g., 'Group A', 'Group B').
    Dependent variable is the effect, should be numerical (e.g., 'Score', 'Conversion Rate').
    '''

    # --- SETUP ---
    # Ensure there are exactly two groups
    groups = df[independent_var_df_col].unique()
    if len(groups) != 2:
        raise ValueError(f"Independent variable '{independent_var_df_col}' must have exactly two groups for a t-test. Found: {len(groups)}")
    
    # break these up into two groups
    group1_name, group2_name = groups[0], groups[1]
    group1_data = df[df[independent_var_df_col] == group1_name][dependent_var_df_col].dropna()
    group2_data = df[df[independent_var_df_col] == group2_name][dependent_var_df_col].dropna()

    # Titles for plots and print statements
    dependent_var_title = dependent_var_df_col.replace('_', ' ').title()
    independent_var_title = independent_var_df_col.replace('_', ' ').title()

    # --- 1. DESCRIPTIVE STATISTICS ---
    print("=== DESCRIPTIVE STATISTICS ===")
    print("\n== Variables: ==")
    print(f"  → Independent Variable (cause): {independent_var_df_col} ('{independent_var_title}')")
    print(f"  → Dependent Variable (effect):  {dependent_var_df_col} ('{dependent_var_title}')")

    print("\n== Group Means, Standard Deviations, N-tiles: ==")
    desc_stats = df.groupby(independent_var_df_col)[dependent_var_df_col].describe()
    print(desc_stats)

    # --- 2. VISUALIZATIONS ---
    print("\n=== VISUALIZATIONS ===")
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f"Analysis of '{dependent_var_title}' by '{independent_var_title}'", fontsize=16)

    # Box plot
    sns.boxplot(data=df, x=independent_var_df_col, y=dependent_var_df_col, ax=axes[0], hue=independent_var_df_col)
    axes[0].set_title(f'Distribution by Group')
    axes[0].set_xlabel(independent_var_title)
    axes[0].set_ylabel(dependent_var_title)
    axes[0].grid(True, alpha=0.3)

    # Violin plot
    sns.violinplot(data=df, x=independent_var_df_col, y=dependent_var_df_col, ax=axes[1], hue=independent_var_df_col)
    axes[1].set_title(f'Distribution Shape by Group')
    axes[1].set_xlabel(independent_var_title)
    axes[1].set_ylabel(dependent_var_title)
    axes[1].grid(True, alpha=0.3)
    
    # Overlapping Histogram
    axes[2].hist(group1_data, alpha=0.7, label=group1_name, bins=15, edgecolor='black')
    axes[2].hist(group2_data, alpha=0.7, label=group2_name, bins=15, edgecolor='black')
    axes[2].set_title(f'Overlapping Histograms')
    axes[2].set_xlabel(dependent_var_title)
    axes[2].set_ylabel('Frequency')
    axes[2].grid(True, alpha=0.3)
    axes[2].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    # --- 3. CHECK ASSUMPTIONS ---
    print("\n=== ASSUMPTION CHECKS ===")

    # A. Normality test (Shapiro-Wilk for each group)
    print("\n== Normality Tests (Shapiro-Wilk): ==")
    normality_results = {}
    for name, data in [(group1_name, group1_data), (group2_name, group2_data)]:
        stat, p_value = stats.shapiro(data)
        normality_results[name] = p_value > 0.05
        print(f"  → Group '{name}': W={stat:.4f}, p={p_value:.4f}", end="")
        if normality_results[name]:
            print(" → Appears normally distributed (p > 0.05) ✅")
        else:
            print(" → May NOT be normally distributed (p ≤ 0.05) ❌")
    
    all_normal = all(normality_results.values())

    # B. Homogeneity of variance (Levene's test)
    print(f"\n== Levene's Test for Equal Variances: ==")
    levene_stat, levene_p = stats.levene(group1_data, group2_data)
    homogeneity = levene_p > 0.05
    print(f"  → Statistic={levene_stat:.4f}, p={levene_p:.4f}", end="")
    if homogeneity:
        print(" → Variances appear equal (p > 0.05) - Assumption met ✅")
    else:
        print(" → Variances may be unequal (p ≤ 0.05) - Use Welch's t-test ❌")

    # --- 4. PERFORM STATISTICAL TESTS ---
    print("\n=== STATISTICAL TESTS ===")

    # A. Test Selection
    parametric = all_normal and homogeneity
    print("\n== Test Selection ==")
    print(f"  → Normality assumption met for all groups: {all_normal}")
    print(f"  → Homogeneity of variance assumption met: {homogeneity}")
    if parametric:
        print("  → Conclusion: Assumptions for parametric test (Student's t-test) are met. ✅")
    else:
        print("  → Conclusion: Assumptions for standard parametric test are NOT fully met. ❌")
        if not homogeneity:
            print("    - Welch's t-test will be used due to unequal variances.")
        if not all_normal:
            print("    - Mann-Whitney U test is the recommended non-parametric alternative.")

    # B. Perform T-Test (Student's or Welch's)
    parametric_icon = "✅" if parametric else "⚠️"
    test_type = "Student's T-Test" if homogeneity else "Welch's T-Test"
    print(f"\n== {parametric_icon} Parametric Test: {test_type} Results ==")
    
    # Use equal_var=False for Welch's t-test if variances are unequal
    t_stat, t_p = stats.ttest_ind(group1_data, group2_data, equal_var=homogeneity)

    print(f"  → T-statistic={t_stat:.4f}, p-value={t_p:.4f}")
    if t_p < 0.05:
        print(f"  → There IS a statistically significant difference between the means of the two groups (p < 0.05) ✅")
    else:
        print(f"  → There is NOT a statistically significant difference between the means of the two groups (p ≥ 0.05) ❌")

    # C. Perform Mann-Whitney U Test (non-parametric alternative)
    non_parametric_icon = "✅" if not all_normal else "⚠️"
    print(f"\n== {non_parametric_icon} Non-Parametric Test: Mann-Whitney U Results ==")
    
    u_stat, u_p = stats.mannwhitneyu(group1_data, group2_data, alternative='two-sided')
    
    print(f"  → U-statistic={u_stat:.4f}, p-value={u_p:.4f}")
    if u_p < 0.05:
        print(f"  → There IS a statistically significant difference between the distributions of the two groups (p < 0.05) ✅")
    else:
        print(f"  → There is NOT a statistically significant difference between the distributions of the two groups (p ≥ 0.05) ❌")

    # --- 5. EFFECT SIZES ---
    print("\n=== EFFECT SIZES ===")

    # A. Cohen's d for T-Test
    n1, n2 = len(group1_data), len(group2_data)
    s1, s2 = np.var(group1_data, ddof=1), np.var(group2_data, ddof=1)
    pooled_std = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    mean_diff = np.mean(group1_data) - np.mean(group2_data)
    cohens_d = mean_diff / pooled_std
    
    print(f"\n== {parametric_icon} Cohen's d (strength of difference between means) ==")
    print(f"  → Effect Size (d): {cohens_d:.4f}")
    d_interpretation = interpret_cohens_d(cohens_d)
    print(f"  → Effect size interpretation: {d_interpretation.upper()}")

    # C. Interpret Cohen's d
    print(f"\n== {parametric_icon} Cohen's d Interpretation based on Statistical Power 0.8 ==")
    print(f"  → {interpret_effect_size_cohens(cohens_d)}")

    # B. Rank-Biserial Correlation for Mann-Whitney U
    rank_biserial = 1 - (2 * u_stat) / (n1 * n2)
    
    print(f"\n== {non_parametric_icon} Rank-Biserial Correlation (strength of difference between distributions) ==")
    print(f"  → Effect Size (r): {rank_biserial:.4f}")
    r_interpretation = interpret_rank_biserial(rank_biserial)
    print(f"  → Effect size interpretation: {r_interpretation.upper()}")
    print("\n" + "="*50 + "\n")



In [ ]:
def interpret_effect_size( effect_size: float) -> str:
    """
    Interpret the effect size based on common thresholds.
    Small Effect: = 0.01
    Medium Effect: = 0.06
    Large Effect: = 0.14

    Args:
        effect_size (float): The calculated effect size value.

    Returns:
        str: Interpretation of the effect size.
    """
    if effect_size < 0.01:
        return "insignificant"
    elif effect_size < 0.06:
        return "small"
    elif effect_size < 0.14:
        return "medium"
    else:
        return "large"
    
def calculate_eta_squared(df: pd.DataFrame, independent_var_col: str, dependent_var_col: str) -> float:
    """
    Calculates the eta-squared effect size for a one-way ANOVA.

    Eta-squared (η²) is a measure of the proportion of variance in the dependent
    variable that is associated with the independent variable.

    Inputs:
        df: A pandas DataFrame containing the data.
        dependent_var_col: The name of the column for the continuous dependent variable.
        independent_var_col: The name of the column for the categorical independent variable (groups).

    Returns:
        The eta-squared value as a float. Returns np.nan if calculations fail.
    """
    try:
        # --- Step 1: Calculate the Overall Mean ---
        # This is the overall mean of the dependent variable for all groups combined.
        grand_mean = df[dependent_var_col].mean()

        # --- Step 2: Calculate the Total Sum of Squares (SS_total) ---
        # This measures the total variation in the data. It's the sum of the
        # squared differences between each individual data point and the grand mean.
        ss_total = ((df[dependent_var_col] - grand_mean) ** 2).sum()

        # --- Step 3: Calculate the Sum of Squares Between Groups (SS_between) ---
        # This measures the variation between the different group means and the grand mean.
        ss_between = 0
        # Get the unique group labels from the independent variable column
        groups = df[independent_var_col].unique()
        
        for group in groups:
            # Get the data for the current group
            group_data = df[df[independent_var_col] == group]
            # Calculate the mean for this specific group
            group_mean = group_data[dependent_var_col].mean()
            # Get the number of observations in this group
            n_group = len(group_data)
            # Calculate the squared difference, weighted by group size, and add to the total
            ss_between += n_group * ((group_mean - grand_mean) ** 2)

        # --- Step 4: Calculate Eta-Squared ---
        # The formula is the ratio of the variance explained by the groups
        # to the total variance in the data.
        if ss_total == 0:
            # Avoid division by zero if there is no variance in the data
            return np.nan 
            
        eta_squared = ss_between / ss_total

        return eta_squared

    except (KeyError, TypeError) as e:
        print(f"An error occurred: {e}. Please check your column names and data types.")
        return np.nan
    

def one_way_anova(df: pd.DataFrame, independent_var_df_col: str, dependent_var_df_col: str):
    '''
    Perform a one-way ANOVA on the specified dependent variable grouped by the independent variable.
    The independent variable should be categorical with 2 > categories (otherwise use an independent samples t-test)

    Independent variable is the cause, should be categorical
    Dependent variable is the effect, should be numerical
    '''

    # Titles
    dependent_var_title = dependent_var_df_col.replace('_', ' ').title()
    independent_var_title = independent_var_df_col.replace('_', ' ').title()

    # 1. DESCRIPTIVE STATISTICS
    print("\n=== DESCRIPTIVE STATISTICS ===")
    print("\n==Variables: ==")
    print(f"  → Independent Variable (cause): {independent_var_df_col} '{independent_var_title}'")
    print(f"  → Dependent Variable (effect):  {dependent_var_df_col} '{dependent_var_title}'")


    print("\n==Group Means, Standard Deviations, N-tiles: ==")
    desc_stats = df.groupby(independent_var_df_col)[dependent_var_df_col].describe()
    print(desc_stats)

    # 2. VISUALIZATIONS
    print("\n=== VISUALIZATIONS ===")
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Box plot
    sns.boxplot(data=df, x=independent_var_df_col, y=dependent_var_df_col, ax=axes[0], hue=independent_var_df_col)
    axes[0].set_title(f'Distribution of {independent_var_title} by {dependent_var_title}')
    axes[0].set_xlabel(dependent_var_title)
    axes[0].set_ylabel(independent_var_title)
    axes[0].grid(True, alpha=0.3)

    # Violin plot
    sns.violinplot(data=df, x=independent_var_df_col, y=dependent_var_df_col, ax=axes[1], hue=independent_var_df_col)
    axes[1].set_title(f'Distribution Shape by {dependent_var_title}')
    axes[1].set_xlabel(dependent_var_title)
    axes[1].set_ylabel(independent_var_title)
    axes[1].grid(True, alpha=0.3)
    
    # Histogram
    for category in df[independent_var_df_col].unique():
        subset = df[df[independent_var_df_col] == category][dependent_var_df_col]
        axes[2].hist(subset, alpha=0.7, label=category, bins=15, edgecolor='black')
    axes[2].set_title(f'Histograms by {dependent_var_title}')
    axes[2].set_xlabel(independent_var_title)
    axes[2].set_ylabel('Frequency')
    axes[2].set_xlim(df[dependent_var_df_col].min(), df[dependent_var_df_col].max())
    axes[2].grid(True, alpha=0.3)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

    # 3. CHECK ASSUMPTIONS FOR ANOVA

    # A. Normality test (Shapiro-Wilk for each group)
    print("\n=== ASSUMPTION CHECKS ===")
    print("\n== Normality Tests (Shapiro-Wilk): ==")
    print("Categories:")
    for category in df[independent_var_df_col].unique():
        group_data = df[df[independent_var_df_col] == category][dependent_var_df_col].dropna()
        stat, p_value = stats.shapiro(group_data)
        print(f"  → {category}: W={stat:.4f}, p={p_value:.4f}", end ="")
        normality = p_value > 0.05
        if normality:
            print(f"  → {category} appears normally distributed (p > 0.05) ✅")
        else:
            print(f"  → {category} may NOT be normally distributed (p ≤ 0.05) ❌")

    # B. Homogeneity of variance (Levene's test)
    print(f"\n== Levene's Test for Equal Variances: ==")
    groups = []
    for category in df[independent_var_df_col].unique():
        group_data = df[df[independent_var_df_col] == category][dependent_var_df_col].dropna()
        groups.append(group_data)

    levene_stat, levene_p = stats.levene(*groups)
    print(f"  → Statistic={levene_stat:.4f}, p={levene_p:.4f}", end="")
    homogeneity = levene_p > 0.05
    if homogeneity:
        print("  → Variances appear equal (p > 0.05) - ANOVA assumption met ✅")
    else:
        print("  → Variances may be unequal (p ≤ 0.05) - Consider Welch's ANOVA ❌")
    print()

    # 4. PERFORM STATISTICAL TESTS
    print("\n=== STATISTICAL TESTS ===")

    # A. TEST SELECTION
    parametric = homogeneity and normality
    parametric_icon = "✅" if parametric else "❌"
    non_parametric_icon = "❌" if parametric else "✅"
    print("\n== Test Selection ==")
    print(f"  → parametric={parametric}, homogenity={homogeneity} → ", end="")
    if parametric:
        print(f" Assumptions for parametric tests (ANOVA) are met. ✅")
    else:
        print("Assumptions for parametric tests (ANOVA) are NOT met. ❌")


    # A. PERFORM ONE-WAY ANOVA
    print(f"\n== {parametric_icon} Parametric Test: One-Way ANOVA Results ==")

    # One way anova, compares the means of groups
    f_stat, anova_p = stats.f_oneway(*groups)

    print(f"  → F-statistic={f_stat:.4f}, p={anova_p:.4f}", end="")

    if anova_p < 0.05:
        print(f"  → Independent variable '{independent_var_title}' HAS a statistically significant effect on the dependent variable '{dependent_var_title}' (p < 0.05) ✅")
        print("   → Post-hoc tests needed to determine which groups differ")
        posthoc = True 
    else:
        print(f"  → Independent variable '{independent_var_title}' DOES NOT HAVE a statistically significant effect on the dependent variable '{dependent_var_title}' (p ≥ 0.05) ❌")
        posthoc = False


    print(f"\n== {non_parametric_icon} Non-Parametrics: Kruskal-Wallis Results ==")

    # Kruskal-Wallis test (non-parametric alternative) compares the median ranks of groups
    kw_stat, kw_p = stats.kruskal(*groups)
    
    print(f"  → H-statistic={kw_stat:.4f}, p={kw_p:.4f}", end="")

    if kw_p < 0.05:
        print(f"  → Independent variable '{independent_var_title}' HAS a statistically significant effect on the dependent variable '{dependent_var_title}' (p < 0.05) ✅")
        print("   → Post-hoc tests needed to determine which groups differ")
        posthoc = True 
    else:
        print(f"  → Independent variable '{independent_var_title}' DOES NOT HAVE a statistically significant effect on the dependent variable '{dependent_var_title}' (p ≥ 0.05) ❌")
        posthoc = False

    # 5. EFFECT SIZE (Eta-squared)

  
    print("\n=== EFFECT SIZES ===")

    # A. Calculate eta-squared (proportion of variance explained)
    eta_squared = calculate_eta_squared(df, independent_var_df_col, dependent_var_df_col)
    print(f"\n== {parametric_icon} Eta-Squared (η²) strength of relationship (ANOVA) ==")
    print(f"  → Effect Size (η²): {eta_squared:.4f}", end="")
    eta_squared_effect_size = interpret_effect_size(eta_squared)
    print(f" → Effect size interpretation: {eta_squared_effect_size.upper()}")
    print(f"    → Approximately {eta_squared * 100:.2f}% of the variance in '{dependent_var_title}' can be explained by '{independent_var_title}'.")


    # B. Calculate Epsilon Squared
    epsilon_squared = kw_stat / (len(df) - 1)
    print(f"\n== {non_parametric_icon} Epsilon-Squared (ε²) proportion of variance (Kruskal-Wallis) ==")
    print(f"  → Effect Size (ε²): {epsilon_squared:.4f}", end="")
    epsilon_squared_effect_size = interpret_effect_size(eta_squared)
    print(f" → Effect size interpretation: {epsilon_squared_effect_size.upper()}")
    print(f"    → Approximately {epsilon_squared * 100:.2f}% of the variance in '{dependent_var_title}' can be explained by '{independent_var_title}'.")


    # C. Convert to Cohen's F  (for >= 3 groups)
    if parametric:
        source_squared = eta_squared
    else:
        source_squared = epsilon_squared

    cohens_f_squared = source_squared / (1 - source_squared)
    cohens_f = np.sqrt(cohens_f_squared)
    print(f"\n== {parametric_icon} Cohen's F (f²) effect size ==")
    print(f"  → Effect Size (f²): {cohens_f:.4f}", end="")
    cohens_f_effect_size = interpret_effect_size_cohens(cohens_f)
    print(f" → Effect size interpretation with respect to power: {cohens_f_effect_size.upper()}")
    print(f"    → Approximately {cohens_f * 100:.2f}% of the variance in '{dependent_var_title}' can be explained by '{independent_var_title}'.")

    
